In [135]:
import open_clip

import torch
from PIL import Image
import random
import glob
import os
import numpy as np
import time
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

import mord


from imblearn.over_sampling import SMOTE

In [147]:
labels

[[1960, 3.0],
 [1960, 3.0],
 [1930, 0.0],
 [1970, 4.0],
 [1960, 3.0],
 [1970, 4.0],
 [1970, 4.0],
 [1950, 2.0],
 [1970, 4.0],
 [1970, 4.0],
 [1930, 0.0],
 [1960, 3.0],
 [1930, 0.0],
 [1950, 2.0],
 [1970, 4.0],
 [1970, 4.0],
 [1930, 0.0],
 [1940, 1.0],
 [1940, 1.0],
 [1940, 1.0],
 [1960, 3.0],
 [1960, 3.0],
 [1950, 2.0],
 [1930, 0.0],
 [1940, 1.0],
 [1940, 1.0],
 [1950, 2.0],
 [1970, 4.0],
 [1950, 2.0],
 [1930, 0.0],
 [1970, 4.0],
 [1970, 4.0],
 [1970, 4.0],
 [1960, 3.0],
 [1960, 3.0],
 [1960, 3.0],
 [1960, 3.0],
 [1950, 2.0],
 [1930, 0.0],
 [1930, 0.0],
 [1970, 4.0],
 [1960, 3.0],
 [1950, 2.0],
 [1930, 0.0],
 [1930, 0.0],
 [1950, 2.0],
 [1950, 2.0],
 [1940, 1.0],
 [1930, 0.0],
 [1940, 1.0],
 [1970, 4.0],
 [1960, 3.0],
 [1950, 2.0],
 [1950, 2.0],
 [1930, 0.0],
 [1930, 0.0],
 [1970, 4.0],
 [1970, 4.0],
 [1960, 3.0],
 [1950, 2.0],
 [1930, 0.0],
 [1960, 3.0],
 [1930, 0.0],
 [1960, 3.0],
 [1950, 2.0],
 [1930, 0.0],
 [1970, 4.0],
 [1960, 3.0],
 [1950, 2.0],
 [1940, 1.0],
 [1950, 2.0],
 [1930

In [5]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [6]:
os.makedirs('./models/', exist_ok=True) 

## Load model

In [7]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32-quickgelu', pretrained='laion400m_e32')
tokenizer = open_clip.get_tokenizer('ViT-B-32-quickgelu')

In [8]:
def get_image_embedding(image_path):
    image = Image.open(image_path)
    tensor = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(tensor)
    return embedding.flatten().cpu().numpy() 

## prepare dataset

In [9]:
imglist = glob.glob('./data/HistoricalColor-ECCV2012/data/imgs/decade_database/**/*.jpg')
random.shuffle(imglist) 

In [91]:
X = []
labels = []
labels_ = []
for img in imglist:
    embedding = get_image_embedding(img)
    #features = np.concatenate((embedding1, embedding2))
    label = int(img.split('/')[6][:-1])
    index = (label - 1930) / 10
    X.append(embedding)
    labels.append([label, index])
    labels_.append(label)


In [92]:
from sklearn.preprocessing import StandardScaler

In [95]:
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

In [108]:


enc = OrdinalEncoder()
y = enc.fit_transform(labels)
y_ = y.ravel()

In [112]:

le = LabelEncoder().fit(ranks)
y = np.array(ranks).astype('i')

In [113]:
ranks = np.array([int(label[1]) for label in labels])
X = np.array(X)

In [136]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, shuffle=True)

In [138]:
oversample = SMOTE()

X_train, y_train = oversample.fit_resample(X_train, y_train)

In [139]:
clf = mord.LogisticIT(alpha=.1, verbose=0)
clf.fit(X_train, y_train)


LogisticIT(alpha=0.1)

In [140]:
y_pred = clf.predict(X_test)

score =accuracy_score(y_test,y_pred)
print(score)

0.42462311557788945


In [131]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88        53
           1       0.38      0.50      0.43        46
           2       0.23      0.22      0.23        50
           3       0.36      0.28      0.31        57
           4       0.45      0.46      0.45        59

    accuracy                           0.46       265
   macro avg       0.46      0.47      0.46       265
weighted avg       0.46      0.46      0.46       265



## Saving model

In [146]:
timestamp = time.strftime("%Y%m%d-%H%M")

with open(f'./models/{timestamp}_ordinal.pkl', 'wb') as f:
    pickle.dump(clf, f)
print('model saved!')

model saved!
